In [67]:
data_str = ['''Conventional wisdom dictates that neural networks take huge amounts training data to reach high performance. Indeed that appears to be the case when training networks from scratch, state of the art image recognition models are typically trained on ImageNet, a dataset of 1.2 million images painstakingly labelled into 1000 classes. It’s definitely possible to train an image classifier for many common objects if enough effort is spent on data collection, however obtaining 1000's of images for every conceivable variation of object is clearly impossible. The same problem holds for a speaker identification system — we simply cannot collect large amounts of labelled audio from every person in the world.

Unlike neural nets, humans are clearly capable of learning quickly from few examples. After seeing a new animal once would you be able to recognize it again afterwards? After talking to a new colleague for a few minutes would you be able to identify their voice the next day? Neural nets have a low sample efficiency compared to humans i.e. they need a lot of data to achieve good performance. Developing ways to improve the sample efficiency of learning algorithms is an area of active research and there are already promising approaches including transfer learning and meta learning among others.

These approaches are based on the intuition that learning a new task/classification problem should be easier once we have already learnt to perform many previous tasks/classification problems as we can leverage previous knowledge. In fact only untrained neural nets have such an absurdly low sample efficiency as it takes a lot of data to shape an untrained neural network (which is just a collection of arrays filled with random numbers) into something useful.

During training, image classifiers learn a hierarchy of useful features: first edge and colour detectors, then more complex shapes and textures. Finally, neurons deep in a network may have a correspondence to quite high level visual concepts (there is a wealth of work exploring this, including this brilliant publication). As an already trained neural network is already able to generate useful, discriminative features, it stands to reason that one should be able to leverage this to quickly adapt to previously unseen classes.''']

In [68]:
num_words = 0
for line in data_str:
    words = line.split()
    num_words += len(words)
print("Number of words:")
print(num_words)

Number of words:
364


In [69]:
target_names = ['com.algo','misc.applications','misc.speaker']

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 200)

In [71]:
from sklearn.datasets import fetch_20newsgroups

In [72]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [73]:
# data pre processing

news_df = pd.DataFrame({'document':data_str})

# removing everything except alphabets`
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# removing short words
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# make all text lowercase
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [74]:
news_df['clean_doc']

0    conventional wisdom dictates that neural networks take huge amounts training data reach high performance indeed that appears case when training networks from scratch state image recognition models...
Name: clean_doc, dtype: object

In [75]:
# removing stop words

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# tokenization
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 37, # keep top 1000 terms 

smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape # check shape of the document-term matrix

(1, 37)

In [77]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

/home/gaourab/anaconda3/lib/python3.5/site-packages/sklearn/decomposition/truncated_svd.py:191: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


1

In [88]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
#     print(sorted_terms)
    
    for i, t in enumerate(sorted_terms):
        print("Topic "+str(i)+": {} {}".format(t[0], t[1]))
#         print(t[0])
#         print(" ")

Topic 0: neural 0.385694607919935
Topic 1: learning 0.3214121732666125
Topic 2: able 0.25712973861329
Topic 3: data 0.25712973861328997
Topic 4: efficiency 0.1928473039599675
Topic 5: image 0.1928473039599675
Topic 6: nets 0.1928473039599675


In [79]:
import umap.umap_ as umap

X_topics = svd_model.fit_transform(X)
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(X_topics)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 
c = dataset.target,
s = 10, # size
edgecolor='none'
)
plt.show()

ImportError: No module named 'umap.umap_'